In [ ]:
#import the library
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import GProfiler
import seaborn as sns
import rpy2.rinterface_lib.callbacks
import logging
import tensorflow as tf
import os

from rpy2.robjects import pandas2ri
import anndata2ri

import importlib
import warnings
warnings.filterwarnings("ignore")
import pickle as pkl
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
#This pallete is for colorblinds
my_palette = ['#0351A8','#8CB0E0','#D56D11','#FFBB78','#234E08','#53CB8B','#D30083','#CB788D','#4E195A','#C58CCF','#AA290F','#B03FD1','#E8BCCF','#64605F','#B2AD9A','#D2D30B','#D1BD4F','#06DCF2','#9EDAE5','#517219','#5B43CF','#D92F24','#FFD900','#002F33','#B8A3A3']

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
values = [0,1]
colors = [(227, 227, 227), (255, 42, 18)]
norm = plt.Normalize(min(values), max(values))
my_cmap = LinearSegmentedColormap.from_list(
    '', [(norm(value), tuple(np.array(color) / 255)) for value, color in zip(values, colors)])

In [ ]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

In [ ]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
.libPaths(.libPaths('C:\\Users\\16220\\AppData\\Local\\R\\win-library\\4.3'))

# Load all the R libraries we will be using in the notebook
library(scran)
library(Seurat)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(ggplot2)
library(plyr)
library(MAST)
library(clusterExperiment)
library(monocle3)
library(SeuratWrappers)
library(magrittr) # needs to be run every time you start R and want to use %>%
library(dplyr)    # alternatively, this also loads %>%

#### For quick loading

In [ ]:
adata =sc.read_h5ad('GSE178318_processed.h5ad')

#### Load the R data we just got

In [ ]:
%%R 
srat_combined = readRDS('processed_GSE178318.rds')

In [ ]:
%%R -o counts -o features -o obs_names
counts = srat_combined@assays$RNA@layers$counts
features = rownames(srat_combined)
obs_names = colnames(srat_combined)

In [ ]:
import anndata
adata = anndata.AnnData(X = counts.T)
adata.var_names = features
adata.obs_names = obs_names

In [ ]:
adata.write('F:/CRC/GSE178318/GSE178318.h5ad')

In [ ]:
sc.pp.normalize_total(adata, key_added = 'normalization_factors')
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='seurat', n_top_genes=3000)
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(adata, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(adata, resolution = 0.8, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(adata)

In [ ]:
sc.pl.tsne(adata, color = 'TRDC')

#### Those clusters seems worth further subclustering

In [ ]:
potential_gd = adata[adata.obs['leiden'].isin(['0','2','5','6','12','19','23']),:]

### 0

In [ ]:
potential_gd = adata[adata.obs['leiden'].isin(['0']),:]

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 50,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden', ['7']), resolution = 0.7, key_added= 'leiden1')
sc.pl.tsne(potential_gd, color=['leiden1','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
gd1 = potential_gd[potential_gd.obs['leiden1'].isin(['7,1']),:]

### 2

In [ ]:
potential_gd = adata[adata.obs['leiden'].isin(['2']),:]

In [ ]:
sc.pp.highly_variable_genes(potential_gd, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_gd, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_gd)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_gd, color=['leiden','CD3E','CD3G','CD3D','TRDC','ICOS','CD4','CD8A','CD8B','FOXP3','IL17A','RORC'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden', ['1']), resolution = 0.5, key_added= 'leiden1')
sc.pl.tsne(potential_gd, color=['leiden1','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden1', ['2']), resolution = 1, key_added= 'leiden2')
sc.pl.tsne(potential_gd, color=['leiden2','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden2', ['2,5']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_gd, color=['leiden3','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden3', ['3']), resolution = 0.5, key_added= 'leiden4')
sc.pl.tsne(potential_gd, color=['leiden4','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden4', ['5']), resolution = 0.5, key_added= 'leiden5')
sc.pl.tsne(potential_gd, color=['leiden5','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden5', ['7']), resolution = 0.3, key_added= 'leiden6')
sc.pl.tsne(potential_gd, color=['leiden6','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
gamma_genesmask = [gene.startswith("TRG") for gene in potential_gd.var_names]
gamma_genes = potential_gd.var_names[gamma_genesmask]
delta_genesmask = [gene.startswith("TRD") for gene in potential_gd.var_names]
delta_genes = potential_gd.var_names[delta_genesmask]

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}
sc.pl.dotplot(potential_gd,TCR_exp_set,groupby = 'leiden6' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd2 = potential_gd[potential_gd.obs['leiden6'].isin(['1,2','2,3','2,5,1','5,0','7,1','7,2']),:]

In [ ]:
sc.pl.dotplot(gd2,TCR_exp_set,groupby = 'leiden6' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

### 5

In [ ]:
potential_gd = adata[adata.obs['leiden'].isin(['5']),:]

In [ ]:
sc.pp.highly_variable_genes(potential_gd, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_gd, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_gd)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_gd, color=['leiden','CD3E','CD3G','CD3D','TRDC','ICOS','CD4','CD8A','CD8B','FOXP3','IL17A','RORC'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden', ['2']), resolution = 1, key_added= 'leiden1')
sc.pl.tsne(potential_gd, color=['leiden1','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden1', ['3']), resolution = 1, key_added= 'leiden2')
sc.pl.tsne(potential_gd, color=['leiden2','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden2', ['7']), resolution =0.8, key_added= 'leiden3')
sc.pl.tsne(potential_gd, color=['leiden3','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden3', ['8']), resolution = 0.8, key_added= 'leiden4')
sc.pl.tsne(potential_gd, color=['leiden4','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
gamma_genesmask = [gene.startswith("TRG") for gene in potential_gd.var_names]
gamma_genes = potential_gd.var_names[gamma_genesmask]
delta_genesmask = [gene.startswith("TRD") for gene in potential_gd.var_names]
delta_genes = potential_gd.var_names[delta_genesmask]

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}
sc.pl.dotplot(potential_gd,TCR_exp_set,groupby = 'leiden4' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd3 = potential_gd[potential_gd.obs['leiden4'].isin(['2,1','3,10','7,1','7,2','8,3']),:]

In [ ]:
sc.pl.dotplot(gd3,TCR_exp_set,groupby = 'leiden4' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

### 6

In [ ]:
potential_gd = adata[adata.obs['leiden'].isin(['6']),:]

In [ ]:
sc.pp.highly_variable_genes(potential_gd, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_gd, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_gd)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_gd, color=['leiden','CD3E','CD3G','CD3D','TRDC','ICOS','CD4','CD8A','CD8B','FOXP3','IL17A','RORC'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden', ['3']), resolution = 1, key_added= 'leiden1')
sc.pl.tsne(potential_gd, color=['leiden1','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden1', ['4']), resolution = 0.9, key_added= 'leiden2')
sc.pl.tsne(potential_gd, color=['leiden2','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden2', ['7']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_gd, color=['leiden3','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
gamma_genesmask = [gene.startswith("TRG") for gene in potential_gd.var_names]
gamma_genes = potential_gd.var_names[gamma_genesmask]
delta_genesmask = [gene.startswith("TRD") for gene in potential_gd.var_names]
delta_genes = potential_gd.var_names[delta_genesmask]

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G'],'others': ['ZBTB16','IKZF2','CD4','CD8A']
}
sc.pl.dotplot(potential_gd,TCR_exp_set,groupby = 'leiden3' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd4 = potential_gd[potential_gd.obs['leiden3'].isin(['4,0','4,5','4,8','7,0','3,1']),:]

In [ ]:
sc.pl.dotplot(gd4,TCR_exp_set,groupby = 'leiden3' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

### 12

In [ ]:
potential_gd = adata[adata.obs['leiden'].isin(['12']),:]

In [ ]:
sc.pp.highly_variable_genes(potential_gd, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_gd, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_gd)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_gd, color=['leiden','CD3E','CD3G','CD3D','TRDC','ICOS','CD4','CD8A','CD8B','FOXP3','IL17A','RORC'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden', ['6']), resolution = 0.9, key_added= 'leiden1')
sc.pl.tsne(potential_gd, color=['leiden1','CD3E','CD3G','CD3D','TRDC'], size = 40, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden1', ['5']), resolution = 0.9, key_added= 'leiden2')
sc.pl.tsne(potential_gd, color=['leiden2','CD3E','CD3G','CD3D','TRDC'], size = 40, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
gamma_genesmask = [gene.startswith("TRG") for gene in potential_gd.var_names]
gamma_genes = potential_gd.var_names[gamma_genesmask]
delta_genesmask = [gene.startswith("TRD") for gene in potential_gd.var_names]
delta_genes = potential_gd.var_names[delta_genesmask]

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}
sc.pl.dotplot(potential_gd,TCR_exp_set,groupby = 'leiden2' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
gd5 = potential_gd[potential_gd.obs['leiden2'].isin(['5,1','6,4','6,5']),:]

In [ ]:
sc.pl.dotplot(gd5,TCR_exp_set,groupby = 'leiden2' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

### 19

In [ ]:
potential_gd = adata[adata.obs['leiden'].isin(['19']),:]

In [ ]:
sc.pp.highly_variable_genes(potential_gd, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_gd, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_gd)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_gd, color=['leiden','CD3E','CD3G','CD3D','TRDC','ICOS','CD4','CD8A','CD8B','FOXP3','IL17A','RORC'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden', ['0']), resolution = 0.5, key_added= 'leiden1')
sc.pl.tsne(potential_gd, color=['leiden1','CD3E','CD3G','CD3D','TRDC'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_gd, restrict_to = ('leiden1', ['3']), resolution = 0.3, key_added= 'leiden2')
sc.pl.tsne(potential_gd, color=['leiden2','CD3E','CD3G','CD3D','TRDC'], size = 40, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
gamma_genesmask = [gene.startswith("TRG") for gene in potential_gd.var_names]
gamma_genes = potential_gd.var_names[gamma_genesmask]
delta_genesmask = [gene.startswith("TRD") for gene in potential_gd.var_names]
delta_genes = potential_gd.var_names[delta_genesmask]

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}
sc.pl.dotplot(potential_gd,TCR_exp_set,groupby = 'leiden2' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd6 = potential_gd[potential_gd.obs['leiden2'].isin(['0,0','4']),:]

In [ ]:
sc.pl.dotplot(gd6,TCR_exp_set,groupby = 'leiden2' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

### 23

In [ ]:
potential_gd = adata[adata.obs['leiden'].isin(['23']),:]

In [ ]:
sc.pp.highly_variable_genes(potential_gd, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_gd, resolution = 1.5, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_gd)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_gd, color=['leiden','CD3E','CD3G','CD3D','TRDC','ICOS','CD4','CD8A','CD8B','FOXP3','IL17A','RORC'],size = 80,cmap = my_cmap)

In [ ]:
gamma_genesmask = [gene.startswith("TRG") for gene in potential_gd.var_names]
gamma_genes = potential_gd.var_names[gamma_genesmask]
delta_genesmask = [gene.startswith("TRD") for gene in potential_gd.var_names]
delta_genes = potential_gd.var_names[delta_genesmask]

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}
sc.pl.dotplot(potential_gd,TCR_exp_set,groupby = 'leiden' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd7 = potential_gd[potential_gd.obs['leiden6'].isin(['1,2','2,3','2,5,1','5,0','7,1','7,2']),:]

In [ ]:
sc.pl.dotplot(gd7,TCR_exp_set,groupby = 'leiden6' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

### end

In [ ]:
adata_gd = gd1.concatenate(gd2,gd3,gd4,gd5,gd6, batch_key = 'original_cluster', batch_categories=['0','2','5','6','12','19'],join = 'outer',fill_value=0)

In [ ]:
#go back to your raw count, use these ID to get your gdT ready for integration
gd_id = adata_gd.index

In [ ]:
sc.pl.tsne(adata_gd,color = ['RORC','IL17A'],cmap = my_cmap)

### Right, do the same thing for IL17

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['19']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 50,cmap = my_cmap)

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['9']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['6']), resolution = 0.6, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 50, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['7']), resolution = 0.3, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 50, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden2' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_2 = potential_17[potential_17.obs['leiden2'].isin(['6,1','7,0']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['10']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['4']), resolution = 0.6, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['10']), resolution = 0.4, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden2', ['7']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_17, color=['leiden3','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden1' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_3 = potential_17[potential_17.obs['leiden1'].isin(['4,0','12']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['0']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['6']), resolution = 0.7, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['10']), resolution = 0.4, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden2', ['7']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_17, color=['leiden3','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden1' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_4 = potential_17[potential_17.obs['leiden1'].isin(['6,1']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['8']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['6']), resolution = 0.7, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['10']), resolution = 0.4, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden2', ['7']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_17, color=['leiden3','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_5 = potential_17[potential_17.obs['leiden'].isin(['7']),:]

In [ ]:
T17_1 = adata[adata.obs['leiden'].isin(['19']),:]

In [ ]:
#Why all of a sudden this does not work??
adata_17 = T17_1.concatenate(T17_2,T17_3,T17_4,T17_5, batch_key = 'original_cluster', batch_categories=['19','9','10','0','8'],join = 'outer',fill_value=0)
#Same thing here, you save the index and go back to raw
ID_17 = adata_17.index

### Visualize the selection

In [ ]:
adata = sc.read_h5ad('GSE178318_processed.h5ad')
adata_17 = sc.read_h5ad('GSE178318_T17.h5ad')
adata_gd = sc.read_h5ad('GSE178318_gd.h5ad')

In [ ]:
adata.obs['IL17 secreting selected'] = '0'
adata.obs['IL17 secreting selected'][adata.obs_names.isin(adata_17.obs_names)] = '1'

In [ ]:
adata.obs['gdT selected'] = '0'
adata.obs['gdT selected'][adata.obs_names.isin(adata_gd.obs_names)] = '1'

In [ ]:
plt.close()
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['figure.figsize'] = [8,8]
fig = sc.pl.tsne(adata, color=['RORC','IL17A','IL17F','IL17 secreting selected'],
                 size =10, ncols = 2, palette = ['#E3E3E3', '#FF2A12'], cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
fig.savefig('17_selected.png',dpi = 300,bbox_inches='tight') 

In [ ]:
plt.close()
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['figure.figsize'] = [8,8]
fig = sc.pl.tsne(adata, color=['CD3E','CD3D','CD3G','CD247','TRDC','gdT selected'],
                 size =10, ncols = 2, palette = ['#E3E3E3', '#FF2A12'], cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
fig.savefig('gd_selected.png',dpi = 300,bbox_inches='tight') 